In [1]:
import torch
from transformer_lens import HookedTransformer
from transformers import AutoTokenizer

# Check if a GPU is available and set the device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Specify the model ID
model_id = "meta-llama/Meta-Llama-3-8B"

# Load the tokenizer from the Hugging Face library
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with TransformerLens, specifying the torch_dtype as torch.float16 to reduce memory usage
model = HookedTransformer.from_pretrained_no_processing(model_id, torch_dtype=torch.float16, device_map=device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B into HookedTransformer


In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict
from tqdm.notebook import tqdm

result_1 = defaultdict(list)
result_2 = defaultdict(list)
max_tokens = 10
lamb = 0.49

with torch.no_grad():
    for layer in range(0, model.cfg.n_layers):
        test_str_1 = tokenizer('Translate "Mary had a little lamb" into Spanish: María', return_tensors='pt').input_ids[0]
        test_str_2 = tokenizer('Translate "Mary tenía un corderito." into English: Mary', return_tensors='pt').input_ids[0]
        seq_len = max(test_str_1.shape[0], test_str_2.shape[0])
        test_str_1 = torch.cat([torch.tensor([tokenizer.bos_token_id] * (seq_len - test_str_1.shape[0]), dtype=torch.long), test_str_1])
        test_str_2 = torch.cat([torch.tensor([tokenizer.bos_token_id] * (seq_len - test_str_2.shape[0]), dtype=torch.long), test_str_2])

        for tok_i in tqdm(range(max_tokens)):
            if len(result_1[layer]) > 0: test_str_1 = torch.cat([test_str_1, torch.tensor([result_1[layer][-1]], dtype=torch.long)])
            if len(result_2[layer]) > 0: test_str_2 = torch.cat([test_str_2, torch.tensor([result_2[layer][-1]], dtype=torch.long)])

            embed_1 = model.input_to_embed(test_str_1)[0].cpu()
            embed_2 = model.input_to_embed(test_str_2)[0].cpu()

            # embed_1 = embed_1[:, :seq_len]
            # embed_2 = embed_2[:, :seq_len]

            resid_1 = model.forward(embed_1, return_type='logits', start_at_layer=0, stop_at_layer=layer).cpu()
            resid_2 = model.forward(embed_2, return_type='logits', start_at_layer=0, stop_at_layer=layer).cpu()

            resid_comb = (lamb) * resid_1 + (1-lamb) * resid_2

            logits = model.forward(resid_comb, return_type='logits', start_at_layer=layer).cpu()
            # print(logits.shape)
            # print(decode(logits[0].argmax(-1).cpu().tolist()))
            val, idx = logits[0].detach().float().softmax(-1).topk(5)
            # print(val.shape)
            if tok_i == max_tokens - 1:
                print('-' * 40)
                print('Merge at layer', layer)
                print('top 5 predictions:')
                for j in range(5):
                    print(('{:>8}' * idx.shape[0]).format(*[repr(tokenizer.decode([i]))[1:-1] for i in idx[:, j].numpy()]))
                    print(('{:8.2f}' * val.shape[0]).format(*[i for i in val[:, j].numpy()]))

            result_1[layer] += [idx[-1, 0]]
            result_2[layer] += [idx[-1, 1]]


  0%|          | 0/10 [00:00<?, ?it/s]

----------------------------------------
Merge at layer 0
top 5 predictions:
QuestionQuestion       "     had       "       a      un       a  little       " Spanish       .       "     had       a  little       .Translate     had       a  little     .\n    Mary
    0.31    0.17    0.22    0.11    0.36    0.49    0.18    0.08    0.12    0.14    0.18    0.23    0.33    0.80    0.91    0.95    0.41    0.09    0.72    0.76    0.75    0.42    0.54
     def     def Translate    land    Mary     two      un  sister       a      in  French     .\n    Mary     had      al    lamb     .\n    Mary     had      al    lamb       .     Mar
    0.10    0.07    0.08    0.08    0.06    0.07    0.17    0.03    0.08    0.06    0.13    0.08    0.12    0.03    0.02    0.01    0.13    0.06    0.10    0.07    0.19    0.23    0.03
       #       #      \n      's     had  little       a     had       ,       ( English       ,            Mary       a     lit       ,       "    Mary      al    litt       ,    

  0%|          | 0/10 [00:00<?, ?it/s]

----------------------------------------
Merge at layer 1
top 5 predictions:
QuestionQuestion     The       P       "      un     son    lamb       "      is Spanish      \n       "   tenía   tenía      un       c      ve      ja     .\n       "  little    lamb
    0.31    0.24    0.03    0.12    0.45    0.27    0.02    0.94    0.24    0.16    0.18    0.28    0.31    0.47    0.59    0.91    0.91    0.97    0.78    0.34    0.17    0.12    0.32
     def     def       I       "     "\n     una    baby    Lamb       ,      in English     .\n   María   María      tu     una       o   order       o       .     The  Little    Lamb
    0.10    0.09    0.01    0.05    0.06    0.13    0.02    0.01    0.11    0.08    0.12    0.15    0.06    0.10    0.15    0.03    0.02    0.02    0.08    0.22    0.05    0.04    0.17
       #       #     the     Had       ,       a       c     lam       .       (  French       .    Mary      tu      un       a     car     vis      jo      \n    Mary    lamb     bo

  0%|          | 0/10 [00:00<?, ?it/s]

...

----------------------------------------
Merge at layer 29
top 5 predictions:
QuestionQuestion       m       "               a     uch       o       "    into English      \n    Mary     had      un       c      ve     ita     .\n<|end_of_text|>    Mary     had      un
    0.31    0.07    0.01    0.02    0.08    0.07    0.07    0.76    0.67    0.41    0.36    0.59    0.54    0.82    0.82    0.46    0.92    0.52    0.62    0.11    0.41    0.65    0.51
     def     def       c       s      un     her  little    lamb       ,      to Spanish       :       "   tenía     una       o   order     ito       .       "    Mary   tenía       a
    0.10    0.05    0.01    0.01    0.08    0.07    0.06    0.17    0.06    0.25    0.11    0.11    0.32    0.11    0.08    0.17    0.08    0.10    0.14    0.07    0.06    0.07    0.39
       #       #       b       k       "     per     uel     ito     que    from       "       .   Maria     Had       a   peque     aca       o      \n    Mary<|end_of_text|>

  0%|          | 0/10 [00:00<?, ?it/s]

----------------------------------------
Merge at layer 30
top 5 predictions:
Question     ://       c       s       "       a  little       o       "    into English      \n    Mary     had      un       c      ña     ita       "   tenía      un       c      ña
    0.31    0.12    0.01    0.02    0.08    0.19    0.25    0.75    0.64    0.43    0.38    0.58    0.62    0.77    0.85    0.41    0.85    0.38    0.09    0.63    0.62    0.25    0.76
     def     php       B       k             her    orte    lamb       ,      to Spanish       :       "   tenía     una   peque       ñ     .\n    Mary   tiene     una   peque       ñ
    0.10    0.07    0.01    0.02    0.08    0.05    0.03    0.14    0.10    0.23    0.11    0.13    0.25    0.15    0.08    0.16    0.09    0.03    0.07    0.09    0.21    0.22    0.22
       #Question       m    land      un     per  Little     ito     que    from  French       (   Maria   tiene       a       o      ño      ín      de     had       a       o      

  0%|          | 0/10 [00:00<?, ?it/s]

----------------------------------------
Merge at layer 31
top 5 predictions:
Question     php       S       s               a  little       o       "    into English      \n    Mary     had      un       c      ña     ita       "    into English     had       a
    0.31    0.02    0.01    0.02    0.13    0.35    0.23    0.56    0.62    0.40    0.38    0.60    0.58    0.84    0.77    0.37    0.87    0.10    0.11    0.12    0.08    0.17    0.12
     def     The       B       k       "     her    baby    lamb       ,      to Spanish       :       "   tenía     una   peque       ñ     .\n       .       "    Mary       :      un
    0.10    0.01    0.01    0.01    0.09    0.07    0.11    0.14    0.10    0.23    0.11    0.10    0.28    0.08    0.10    0.20    0.05    0.05    0.03    0.06    0.05    0.13    0.05
       #Question       M       a      un     per     uch       "       .    from  French       (   Maria     Had       a     cab      ño      de    Mary Translate       "   tenía    